In [1]:
# import os for creating directories and file paths in the notebook environment
import os

In [2]:
# checking the current working directory of the notebook environment
%pwd

'g:\\Jagadish\\Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS\\research'

In [3]:
# changing the current working directory to the parent directory of the notebook environment
os.chdir("../")

In [4]:
# checking the current working directory of the notebook environment
%pwd

'g:\\Jagadish\\Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    """Configuration for preparing a base model.
    Attributes:
        root_dir (Path): Root directory for the project.
        base_model_path (Path): Path to the base model.
        updated_base_model_path (Path): Path to the updated base model.
        params_image_size (list): List containing image dimensions.
        params_learning_rate (float): Learning rate for the model.
        params_include_top (bool): Whether to include the top layers of the model.
        params_weights (str): Weights to be used for the model.
        params_classes (int): Number of output classes for the model.
    """
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

[2024-12-14 19:49:06,163: INFO: __init__: Logging setup is complete.]


In [7]:
class ConfigurationManager:
    """
    A class to manage configuration files and parameters.

    Attributes:
        config (dict): Configuration settings read from a YAML file.
        params (dict): Parameters settings read from a YAML file.

    Methods:
        get_prepare_base_model_config(): Prepares the configuration for the base model.
    """

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initializes the ConfigurationManager with the configuration and parameters file paths.

        Args:
            config_filepath (str): Path to the configuration YAML file.
            params_filepath (str): Path to the parameters YAML file.
        """
        # Read the configuration and parameters from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create the necessary directories specified in the configuration
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        Prepares the configuration for the base model.

        Reads the necessary settings from the configuration and parameters,
        creates the required directories, and returns a PrepareBaseModelConfig object.

        Returns:
            PrepareBaseModelConfig: Configuration object for preparing the base model.
        """
        config = self.config.prepare_base_model
        
        # Create the root directory specified in the base model configuration
        create_directories([config.root_dir])

        # Prepare the base model configuration using the settings from the configuration and parameters
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    """
    A class to prepare and update the base model using the given configuration.

    Attributes:
        config (PrepareBaseModelConfig): Configuration for preparing the base model.
        model (tf.keras.Model): The base model.
        full_model (tf.keras.Model): The updated full model.

    Methods:
        get_base_model(): Initializes the base model.
        update_base_model(): Updates the base model by adding custom layers and compiling.
        save_model(path, model): Saves the given model to the specified path.
    """
    def __init__(self, config: PrepareBaseModelConfig):
        """
        Initializes the PrepareBaseModel with the given configuration.

        Args:
            config (PrepareBaseModelConfig): Configuration for preparing the base model.
        """
        self.config = config

    def get_base_model(self):
        """
        Loads the base model using the VGG16 architecture with the specified parameters
        and saves it to the configured path.
        """
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        # Save the base model to the specified path
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        """
        Prepares the full model by adding custom layers, setting trainable layers, and compiling.

        Args:
            model (tf.keras.Model): The base model.
            classes (int): Number of output classes.
            freeze_all (bool): Whether to freeze all layers.
            freeze_till (int or None): Number of layers to keep unfrozen from the end.
            learning_rate (float): Learning rate for the optimizer.

        Returns:
            tf.keras.Model: The fully prepared model with custom layers added.
        """
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        """
        Updates the base model by preparing and compiling the full model with custom layers.
        The updated model is then saved to the configured path.
        """
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        # Save the updated full model to the specified path
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the given model to the specified path.

        Args:
            path (Path): The path where the model should be saved.
            model (tf.keras.Model): The model to save.
        """
        model.save(path)


In [10]:
try:
    # Initialize ConfigurationManager with default file paths
    config = ConfigurationManager()
    
    # Get the configuration for preparing the base model
    prepare_base_model_config = config.get_prepare_base_model_config()
    
    # Initialize the PrepareBaseModel with the obtained configuration
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    
    # Load and save the base model
    prepare_base_model.get_base_model()
    
    # Update the base model by adding custom layers and compiling
    prepare_base_model.update_base_model()

except Exception as e:
    # Raise the exception if any error occurs during the process
    raise e


[2024-12-14 20:04:22,471: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-14 20:04:22,486: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-14 20:04:22,611: INFO: common: created directory at: artifacts]
[2024-12-14 20:04:22,619: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 14s 0us/step
[2024-12-14 20:04:44,095: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                